# Using the OpenAI API
* **Created by:** Eric Martinez
* **For:** Software Engineering 2
* **At:** University of Texas Rio-Grande Valley

## OpenAI API

The OpenAI API provides access to powerful LLMs like GPT-3.5 and GPT-4, enabling developers to leverage these models in their applications. To access the API, sign up for an API key on the OpenAI website and follow the documentation to make API calls.

For enterprise: Azure OpenAI offers a robust and scalable platform for deploying LLMs in enterprise applications. It provides features like security, compliance, and support, making it an ideal choice for businesses looking to leverage LLMs.

Options:
* [[Free] Sign-up for access to my OpenAI service](https://platform.openai.com/signup) - _requires your UTRGV email and student ID_
* [[Paid] Alternatively, sign-up for OpenAI API Access](https://platform.openai.com/signup)

## Managing Application Secrets

Secrets are sensitive information, such as API keys, passwords, or cryptographic keys, that must be protected to ensure the security and integrity of a system.

In software development, secrets are often used to authenticate users, grant access to resources, or encrypt/decrypt data. Mismanaging or exposing secrets can lead to severe security breaches and data leaks.

#### Common examples of secrets
* API keys
* Database credentials
* SSH keys
* OAuth access tokens
* Encryption/decryption keys

#### Common mistakes when handling secrets
* Storing secrets in plain text
* Hardcoding secrets in source code
* Sharing secrets through unsecured channels (e.g., email or messaging apps)
* Using the same secret for multiple purposes
* Not rotating or updating secrets regularly

How attackers might obtain secrets
* Exploiting vulnerabilities in software or infrastructure
* Intercepting unencrypted communications
* Gaining unauthorized access to repositories or storage systems
* Social engineering or phishing attacks
* Brute-forcing weak secrets

The impact of compromised secrets
* Unauthorized access to sensitive data
* Data breaches, resulting in financial loss and reputational damage
* Loss of trust from customers and stakeholders
* Legal repercussions and regulatory fines
* Potential takeover or manipulation of systems

Steps to protect secrets
* Store secrets securely using secret management tools or dedicated secret storage services
* Encrypt secrets at rest and in transit
* Use strong, unique secrets and rotate them regularly
* Limit access to secrets on a need-to-know basis
* Implement proper auditing and monitoring of secret usage

Cloud services and secret management
* Many cloud providers offer secret management services (e.g., AWS Secrets Manager, Azure Key Vault, Google Cloud Secret Manager) that securely store, manage, and rotate secrets.
* These services often provide access control, encryption, and auditing capabilities.
* Integrating cloud secret management services with your application can help secure sensitive information and reduce the risk of exposure.

Best practices for secrets
* Use different secrets for development, testing, and production environments to minimize the risk of accidental exposure.
* Regularly audit and review secret access to ensure only authorized users have access.
* Establish a clear process for managing secrets, including secret creation, storage, rotation, and deletion.
* Educate team members on the importance of secret security and the best practices for handling sensitive information.

#### Using  `.dotenv` library to protect secrets in Python

 `.dotenv` is a Python library that allows developers to load environment variables from a `.env` file. It helps keep secrets out of source code and makes it easier to manage and update them.

Install the `python-dotenv` library

In [45]:
!pip -q install --upgrade python-dotenv

Create a `.env` file in this folder using any editor.

Add secrets as key-value pairs in the `.env` file

If you are using my OpenAI service use the following format:

In [ ]:
OPENAI_API_BASE=<my API base>
OPENAI_API_KEY=<your API key to my service>

If you are not using my OpenAI service then use the following format:

In [ ]:
OPENAI_API_KEY=<your OpenAI API key>

Then, use the following code to load those secrets into this notebook:

In [47]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

#### Install Dependencies

In [1]:
!pip -q install --upgrade openai

#### Let's make a function to wrap OpenAI functionality and write some basic tests

Start by simply seeing if we can make an API call

In [50]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

model="gpt-3.5-turbo"
messages=[{"role": "user", "content": "hello"}]

completion = openai.ChatCompletion.create(model=model, messages=messages)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


Great! Now let's wrap that in a function

In [19]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

def get_ai_reply(model="gpt-3.5-turbo", user_message=""):
    messages=[{"role": "user", "content": user_message}]
    completion = openai.ChatCompletion.create(model=model, messages=messages)
    return completion.choices[0].message.content

print(get_ai_reply(user_message="hello"))

Hello! How can I assist you today?


Let's add some tests!

These are traditional type of tests.

Since the output is non-deterministic, generally, what are some things that we could test?

At the very least, maybe that the output is a string?

In [20]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

def get_ai_reply(message, model="gpt-3.5-turbo"):    
    messages=[{"role": "user", "content": message}]
        
    completion = openai.ChatCompletion.create(model=model, messages=messages, temperature=temperature)
    return completion.choices[0].message.content

# traditional tests
assert isinstance(get_ai_reply("hello"), str)
assert isinstance(get_ai_reply("hello", model="gpt-3.5-turbo"), str)

But what if we do what to test the output of the LLM?

Is there anyway to control, atleast to some degree, the level of non-determinism?

Yes! Let's add a temperature parameter, this will help us control the 'creativity' and 'randomness' of the response.

Setting it to 0 helps ensure outputs are more consistent when given the same input.

Valid values for temperature are between 0 and 1, inclusive.

This will help us when writing tests, but is something that we should keep in mind that if we write tests against the LLM output, we might get the expect results _ONLY_ at low temperature.

An ideal test strategy should resemble the temperature setting we will use in production.

In [31]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

def get_ai_reply(message, model="gpt-3.5-turbo", temperature=0):    
    messages=[{"role": "user", "content": message}]
        
    completion = openai.ChatCompletion.create(model=model, messages=messages, temperature=temperature)
    return completion.choices[0].message.content

# traditional tests
assert isinstance(get_ai_reply("hello"), str)
assert isinstance(get_ai_reply("hello", model="gpt-3.5-turbo"), str)

In [ ]:
If we run this enough times we should see that the output for the bottom run is more inconsistent.

In [33]:
print(get_ai_reply("hello")) # uses default of temperature 0
print(get_ai_reply("hello", temperature=0.9)) # uses default of temperature 0

Hello! How can I assist you today?
Hello there! How can I assist you today?


Ok great! Now, an LLM is no good to us if we can't _steer_ it.

So let's add the ability to input a _prompt_ or _system message_.

In [37]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

# traditional tests
assert isinstance(get_ai_reply("hello"), str)
assert isinstance(get_ai_reply("hello", model="gpt-3.5-turbo"), str)

Let's see if we can get the LLM to follow instructions by adding instructions to the prompt.

Run this cell a few times and see what happens. Is it consistent?

In [36]:
print(get_ai_reply("hello", model="gpt-3.5-turbo", system_message="When I say 'hello' you simply reply with 'world.'"))

world.


While the output is more or less controlled, the LLM responds with 'world.' or 'world'. While the word 'world' being in the string is pretty consistent, the punctuation is not.

How do we write tests against this or have confidence with non-determinism?

What is a test that we could write that:
* would pass if the LLM outputs in a manner that is consistent with our expectations (and consistent with its own output)?
* _we want to be true_ about our LLM system, and if it does not then we would want to know immediately and adjust our system?
* if the prompt does not change, that our expectation holds true?
* someone changes the prompt in a way that would break the rest of the system, that we would want to prevent that from being merged without fixing the downstream effects?

In [38]:
# non-deterministic tests
system_message="When I say 'hello' you simply reply with 'world.'"
message="hello"

assert "world" in get_ai_reply(message, system_message=system_message)

Alright that worked!

Now, let's extend the functionality to add the ability to pass message history so that it can have memory about what was said previously.

In [39]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]
    
    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

# traditional tests
assert isinstance(get_ai_reply("hello"), str)
assert isinstance(get_ai_reply("hello", model="gpt-3.5-turbo"), str)

# non-deterministic unit tests
assert "world" in get_ai_reply("hello", model="gpt-3.5-turbo", system_message="When I say 'hello' you simply reply with 'world.'")

Now let's check that it works

In [42]:
system_message="The user will tell you their name. When asked, repeat their name back to them."
message_history = [
   {"role": "user", "content": "My name is Bob."},
   {"role": "assistant", "content": "Nice to meet you, Bob."}
]
message = "What was my name again?"
print(get_ai_reply(message, system_message=system_message, message_history=message_history))

Your name is Bob.


Great! Now let's turn that into a test!

In [43]:
system_message="The user will tell you their name. When asked, repeat their name back to them."
message_history = [
   {"role": "user", "content": "My name is Bob."},
   {"role": "assistant", "content": "Nice to meet you, Bob."}
]
message = "What was my name again?"

assert "Bob" in get_ai_reply(message, system_message=system_message, message_history=message_history)

Alright here is our final function for integrating with OpenAI!

In [ ]:
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

# traditional unit tests
assert isinstance(get_ai_reply("hello"), str)
assert isinstance(get_ai_reply("hello", model="gpt-3.5-turbo"), str)

# non-deterministic unit tests
assert "world" in get_ai_reply("hello", model="gpt-3.5-turbo", system_message="When I say 'hello' you simply reply with 'world.'")

system_message="The user will tell you their name. When asked, repeat their name back to them."
message_history = [
   {"role": "user", "content": "My name is Bob."},
   {"role": "assistant", "content": "Nice to meet you, Bob."}
]
message = "What was my name again?"

assert "Bob" in get_ai_reply(message, system_message=system_message, message_history=message_history)

In [44]:
print(get_ai_reply("hello"))

Hello! How can I assist you today?


In the next few lessons, we will be building a graphical user interface around this functionality so we can have a real conversational experience.